# Process Synchronization Using Barrier Object:

### Barrier Objects for Processes:
- In Python's multiprocessing module, a Barrier is used to synchronize a set of processes, ensuring that they all reach a certain point in their execution before proceeding. The concept is similar to thread synchronization, but it is designed to work with processes instead of threads. This is useful when you need to coordinate multiple processes, such as in parallel computations or when processes depend on each other to complete a task before continuing.

### Methods and Attributes of multiprocessing.Barrier:
#### i. wait(timeout=None):
- Purpose: Used by processes to wait at the barrier.
- How It Works: When a process calls barrier.wait(), it blocks until all participating processes have also called wait(). Once all processes have called wait(), the barrier is released, and all processes can proceed with their execution.
- Timeout: The timeout parameter allows the process to wait for a specific time before raising a TimeoutError if not all processes reach the barrier.
#### ii. abort():
- Purpose: Used to abort the barrier and reset its state.
- How It Works: If a process needs to stop the barrier prematurely, it can call barrier.abort(). This will cause the barrier to reset, and any process that subsequently calls wait() will raise a BrokenBarrierError, signaling that the barrier was aborted.
#### iii. n_waiting:
- Purpose: Provides the number of processes currently waiting at the barrier.
- How It Works: This attribute can be used to check how many processes are waiting to synchronize at the barrier before they can all proceed.
#### iv. parties:
- Purpose: Returns the total number of processes that need to reach the barrier before it is released.
- How It Works: This attribute indicates how many processes must call wait() before the barrier is triggered. It is set when the barrier is initialized and is usually the same number for all participating processes.

## Scenario:
- We have four players playing a game. Each player plays 3 rounds of the game. After completing their rounds, they should wait for all players to finish before they can receive their winning amount. The issue is that without synchronization, players might receive their winnings at different times, potentially causing inconsistent results.

### 1. Solution Without Barrier:
- In this approach, the players will execute independently, without waiting for each other. This could lead to players receiving their winnings before the others have completed their tasks.

In [2]:
import multiprocessing
import time

def player(name):
    print(f"{name} started playing.")

    # Simulate the playing process (a task each player does)
    for i in range(3):
        time.sleep(3)
        print(f"{name} is playing round {i + 1}")

    # After finishing playing, the player immediately sends the winning amount
    print(f"sending winning amount to {name}")

def main():
    # List of players
    players_name = ['susan', 'pratik', 'suman', 'nijan']
    processes = []

    # Create and start processes
    for name in players_name:
        process = multiprocessing.Process(target=player, args=(name,))
        processes.append(process)
        process.start()

    # Wait for all processes to finish
    for process in processes:
        process.join()

    print("All players have received their winning amount.")

if __name__ == "__main__":
    main()


susan started playing.
suman started playing.pratik started playing.

nijan started playing.
susan is playing round 1
pratik is playing round 1
suman is playing round 1
nijan is playing round 1
susan is playing round 2
pratik is playing round 2
suman is playing round 2nijan is playing round 2

susan is playing round 3
sending winning amount to susan
pratik is playing round 3
nijan is playing round 3suman is playing round 3sending winning amount to pratik

sending winning amount to nijansending winning amount to suman


All players have received their winning amount.


### Issues Without Barrier:
- Inconsistent flow: Some players may finish early and get their winnings while others are still playing.
- No synchronization: There's no enforced wait time for all players to finish before the next phase.
- Race conditions: The timing of each player's process execution could lead to unpredictable results.

### 2. Solution With Barrier:
- In this approach, we'll use a Barrier to synchronize the players. This ensures that all players must finish their rounds before they can proceed to the "winning amount" phase. The Barrier will block each player process until all processes reach it, and only then will the processes proceed.

In [3]:
import multiprocessing
import time

def player(name, barrier):
    print(f"{name} started playing.")

    # Simulate the playing process (a task each player does)
    for i in range(3):
        time.sleep(3)
        print(f"{name} is playing round {i + 1}")

    # Synchronize the players: make them wait until all have finished playing
    print(f"{name} finished playing and is waiting for others.")
    barrier.wait()  # All processes will wait at this point

    # After all players are done, they proceed to the next task
    print(f"sending winning amount to {name}")

def main():
    # List of players
    players_name = ['susan', 'pratik', 'suman', 'nijam']
    Processes = []

    # Create a Barrier for 4 players
    barrier = multiprocessing.Barrier(4)

    # Create and start processes
    for name in players_name:
        process = multiprocessing.Process(target=player, args=(name, barrier))
        Processes.append(process)
        process.start()

    # Wait for all processes to finish
    for process in Processes:
        process.join()

    print("All winning players have received their winning amount.")

if __name__ == "__main__":
    main()


susan started playing.
pratik started playing.suman started playing.

nijam started playing.
susan is playing round 1
suman is playing round 1pratik is playing round 1

nijam is playing round 1
susan is playing round 2
pratik is playing round 2
suman is playing round 2
nijam is playing round 2
susan is playing round 3
susan finished playing and is waiting for others.
pratik is playing round 3
pratik finished playing and is waiting for others.
suman is playing round 3
suman finished playing and is waiting for others.
nijam is playing round 3
nijam finished playing and is waiting for others.
sending winning amount to sumansending winning amount to nijamsending winning amount to pratiksending winning amount to susan



All winning players have received their winning amount.


### Advantages of Barrier Object:
- Process Synchronization: Ensures all processes reach a certain point before proceeding.
- Prevents Race Conditions: Eliminates conflicts by coordinating process execution.
- Coordinated Execution: Aligns processes to perform tasks in a specific order.
- Prevents Deadlocks: Avoids situations where processes are stuck waiting on each other.

### Use Cases for Barrier Object:
- Parallel Data Processing: Synchronizing multiple processes that perform portions of a task and need to wait for each other to proceed with the next phase.
- Multi-phase Computations: In algorithms that require all processes to complete one phase before moving on to the next (e.g., simulation models).
- Batch Processing: Ensuring that processes perform their work in synchronization before moving on to the next step.
- Synchronization of Workers: In scenarios where multiple workers need to synchronize at specific points (e.g., map-reduce frameworks).